# V4系列で吐き出されたスコアシートについて分析をするスクリプト

## ゴールとしては，このスクリプトで追加した条件によって，今後収益が出るかどうかを見分けられるルールを決めたい

### モジュールのインポート

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
from sklearn.model_selection import train_test_split
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
import time
import datetime
from dateutil.relativedelta import relativedelta
import os #ディレクトリ作成用
import xgboost as xgb
import sys
#自作のモジュールのインポート
sys.path.append("..")
import module.master as master
import module.modeling_scores as modeling

### 基礎集計

In [2]:
version='V4_2'#バージョン

place_master=master.get_place_master()
place_name='asiya'

#ファイルの読み込み
# for i in range(4):
#     now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1)
#     dir_path = "../../bot_database/{place_name}/model_score_{place_name}/v4_score/{place_name}_model_score_st{stDate}_{V}.csv".format(place_name=place_name,V=version,stDate=now_ym.strftime('%Y%m%d'))#作成したデータの書き込み先#使用するデータの読み込み
#     model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
#     model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
#     display(model_score_df)

#testでのnum_plus_monthの数
print('testでのnum_plus_monthの数====================================================================')
for i in range(4):
    now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1)
    dir_path = "../../bot_database/{place_name}/model_score_{place_name}/v4_score/{place_name}_model_score_st{stDate}_{V}.csv".format(place_name=place_name,V=version,stDate=now_ym.strftime('%Y%m%d'))#作成したデータの書き込み先#使用するデータの読み込み
    model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
    model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
    print('st_date:{}'.format(now_ym))
    print(model_score_df['plus_month_num_test'].value_counts().sort_index())
    



testでのnum_plus_monthの数====================================================================
st_date:2020-01-01 00:00:00
0.0    5409
1.0    5612
2.0    2520
3.0     512
4.0      59
Name: plus_month_num_test, dtype: int64
st_date:2020-04-01 00:00:00
0.0    5105
1.0    5115
2.0    3119
3.0     558
4.0     215
Name: plus_month_num_test, dtype: int64
st_date:2020-07-01 00:00:00
0.0    4850
1.0    4404
2.0    3181
3.0    1571
4.0     106
Name: plus_month_num_test, dtype: int64
st_date:2020-10-01 00:00:00
0.0    5063
1.0    5348
2.0    2682
3.0     894
4.0     120
5.0       5
Name: plus_month_num_test, dtype: int64


In [3]:
#testでのnum_plus_monthが3以上だったもののうち，での,finalのnumplus_monthの数
print('testでのnum_plus_monthが3以上だったもののうち，での,finalのnumplus_monthの数====================================================================')
for i in range(4):
    now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1)
    dir_path = "../../bot_database/{place_name}/model_score_{place_name}/v4_score/{place_name}_model_score_st{stDate}_{V}.csv".format(place_name=place_name,V=version,stDate=now_ym.strftime('%Y%m%d'))#作成したデータの書き込み先#使用するデータの読み込み
    model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
    model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
    print('st_date:{}'.format(now_ym))
    #集計
    print(model_score_df[model_score_df['plus_month_num_test']>=3]['plus_month_num_final'].value_counts().sort_index())

testでのnum_plus_monthが3以上だったもののうち，での,finalのnumplus_monthの数====================================================================
st_date:2020-01-01 00:00:00
0.0      3
1.0     21
2.0    133
3.0    222
4.0    191
5.0      1
Name: plus_month_num_final, dtype: int64
st_date:2020-04-01 00:00:00
0.0    344
1.0    299
2.0    116
3.0      8
4.0      4
5.0      2
Name: plus_month_num_final, dtype: int64
st_date:2020-07-01 00:00:00
0.0    608
1.0    327
2.0    197
3.0    279
4.0    261
5.0      5
Name: plus_month_num_final, dtype: int64
st_date:2020-10-01 00:00:00
0.0    527
1.0    383
2.0    107
3.0      2
Name: plus_month_num_final, dtype: int64


In [4]:
#test,finalでnum_plus_monthが3以上だったモデルの重複なしでのcomたち
print('#test,finalでnum_plus_monthが3以上だったモデルの重複なしでのcomたち====================================================================')
for i in range(4):
    now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1)
    dir_path = "../../bot_database/{place_name}/model_score_{place_name}/v4_score/{place_name}_model_score_st{stDate}_{V}.csv".format(place_name=place_name,V=version,stDate=now_ym.strftime('%Y%m%d'))#作成したデータの書き込み先#使用するデータの読み込み
    model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
    model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
    print('st_date:{}'.format(now_ym))
    #集計
    print(model_score_df[(model_score_df['plus_month_num_test']>=3) & (model_score_df['plus_month_num_final']>=3)]['target_com'].value_counts().sort_index())

#test,finalでnum_plus_monthが3以上だったモデルの重複なしでのcomたち====================================================================
st_date:2020-01-01 00:00:00
1.0     117
2.0      16
5.0      36
6.0      13
7.0      17
10.0     71
14.0    144
Name: target_com, dtype: int64
st_date:2020-04-01 00:00:00
1.0      1
13.0     2
21.0    11
Name: target_com, dtype: int64
st_date:2020-07-01 00:00:00
4.0       1
13.0    262
15.0     82
22.0    200
Name: target_com, dtype: int64
st_date:2020-10-01 00:00:00
1.0    2
Name: target_com, dtype: int64


In [5]:
#test,finalでnum_plus_monthが3以上だったモデルの重複なしでのcomたち
print('#test,finalでnum_plus_monthが3以上だったモデルの重複なしでのcomたち====================================================================')
for i in range(4):
    now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1)
    dir_path = "../../bot_database/{place_name}/model_score_{place_name}/v4_score/{place_name}_model_score_st{stDate}_{V}.csv".format(place_name=place_name,V=version,stDate=now_ym.strftime('%Y%m%d'))#作成したデータの書き込み先#使用するデータの読み込み
    model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
    model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
    print('st_date:{}'.format(now_ym))
    #集計
    print(model_score_df[(model_score_df['gain_test']>=115) & (model_score_df['gain_final']>=115)]['target_com'].value_counts().sort_index())

#test,finalでnum_plus_monthが3以上だったモデルの重複なしでのcomたち====================================================================
st_date:2020-01-01 00:00:00
2.0       8
3.0      41
5.0       8
6.0       3
7.0     181
8.0      33
11.0     23
13.0      1
14.0    274
16.0      1
20.0     32
22.0      2
23.0      2
25.0      9
Name: target_com, dtype: int64
st_date:2020-04-01 00:00:00
1.0       1
4.0      13
12.0      7
21.0     17
22.0     12
23.0    282
25.0     23
27.0     65
Name: target_com, dtype: int64
st_date:2020-07-01 00:00:00
4.0      10
12.0    121
13.0    401
15.0    245
21.0      8
22.0    244
23.0    128
Name: target_com, dtype: int64
st_date:2020-10-01 00:00:00
1.0       2
5.0       5
11.0      1
12.0    147
13.0    244
21.0      6
22.0    252
Name: target_com, dtype: int64


In [6]:
#test,finalでnum_plus_monthが3以上だったモデルの重複なしでのcomたち
def gainth_analysis(score_df,test_type='final'):
    
    gainth_arr=[100,110,120,140,160]
    index_arr=[0,1,2,3,4]
    cols=["gain_th","num_com","num_model(row)"]
    gainth_df=pd.DataFrame(columns=cols)
    def make_series(target_df):
        num_com=len(target_df['target_com'].value_counts().index)
        num_model=len(target_df)
        return num_com,num_model
    for gainth,i in zip(gainth_arr,index_arr):
        append_s=pd.Series(index=cols,dtype='object')
        if (i==0):
            target_df=score_df[score_df["gain_{}".format(test_type)]<gainth]
            num_com,num_model=make_series(target_df)
            append_s['gain_th']="100未満"
            append_s['num_com']=num_com
            append_s['num_model(row)']=num_model
        elif (i==4):
            target_df=score_df[score_df["gain_{}".format(test_type)]>=gainth]
            num_com,num_model=make_series(target_df)
            append_s['gain_th']="150以上"
            append_s['num_com']=num_com
            append_s['num_model(row)']=num_model
        else:
            target_df=score_df[(score_df["gain_{}".format(test_type)]<gainth_arr[i])&(score_df["gain_{}".format(test_type)]>=gainth_arr[i-1])]
            num_com,num_model=make_series(target_df)
            append_s['gain_th']="{}以上_{}未満".format(gainth_arr[i-1],gainth_arr[i])
            append_s['num_com']=num_com
            append_s['num_model(row)']=num_model
        gainth_df=gainth_df.append(append_s,ignore_index=True)
    return gainth_df

print('#testでnum_plus_monthが3以上だったモデルのgain分析====================================================================')
for i in range(4):
    now_ym=datetime.datetime(year=2020, month=1+(i*3),day=1)
    dir_path = "../../bot_database/{place_name}/model_score_{place_name}/v4_score/{place_name}_model_score_st{stDate}_{V}.csv".format(place_name=place_name,V=version,stDate=now_ym.strftime('%Y%m%d'))#作成したデータの書き込み先#使用するデータの読み込み
    model_score_df=pd.read_csv(dir_path, encoding='utf_8_sig')
    model_score_df=model_score_df.drop(["Unnamed: 0"],axis=1)
    print('st_date:{}'.format(now_ym))
    gainth_df=gainth_analysis(model_score_df)
    #集計
    display(gainth_df)
    
    


#testでnum_plus_monthが3以上だったモデルのgain分析====================================================================
st_date:2020-01-01 00:00:00


,gain_th,num_com,num_model(row)
0,100未満,25,6749
1,100以上_110未満,25,1512
2,110以上_120未満,25,1676
3,120以上_140未満,25,1641
4,150以上,18,1987


st_date:2020-04-01 00:00:00


,gain_th,num_com,num_model(row)
0,100未満,28,11414
1,100以上_110未満,18,365
2,110以上_120未満,16,331
3,120以上_140未満,14,533
4,150以上,12,1071


st_date:2020-07-01 00:00:00


,gain_th,num_com,num_model(row)
0,100未満,28,11341
1,100以上_110未満,16,392
2,110以上_120未満,15,228
3,120以上_140未満,15,340
4,150以上,9,1422


st_date:2020-10-01 00:00:00


,gain_th,num_com,num_model(row)
0,100未満,28,10162
1,100以上_110未満,16,736
2,110以上_120未満,13,762
3,120以上_140未満,12,860
4,150以上,12,1124


In [7]:
test_type='final'
model_score_df[(model_score_df["gain_{}".format(test_type)]<gainth_arr[i])&(model_score_df["gain_{}".format(test_type)]>=gainth_arr[i-1])]

NameError: name 'gainth_arr' is not defined